# SigE
Analysis of chip-seq for SigE in EtOH stres vs nostres conditions

In [1]:
import pandas as pd
import os
import re
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from IPython.display import IFrame, display, Image
import shutil
import subprocess
import functools
import operator
from matplotlib import pyplot as plt
import numpy as np
import math
import time
import seaborn as sns
import sys
from Bio import SeqUtils
from itertools import islice      

sns.set()

def chunker(seq, l):
    for i in range(0, len(seq), l):
        yield seq[i:i+l]
        
def window(seq, n=2):
    "Returns a sliding window (of width n) over data from the iterable"
    "   s -> (s0,s1,...s[n-1]), (s1,s2,...,sn), ...                   "
    it = iter(seq)
    result = tuple(islice(it, n))
    if len(result) == n:
        yield result
    for elem in it:
        result = result[1:] + (elem,)
        yield result

In [2]:
temp = 'peak_summit_motif_discovery'
if os.path.exists(temp):
    shutil.rmtree(temp)
os.makedirs(temp, exist_ok=True)
os.environ['JUPYTER_TEMP'] = temp

In [3]:
# Streptomyces genome accession AL645882.2
SC_ACCESSION = "AL645882.2"

sc_genome = [
    s for s in SeqIO.parse('data/GCA_000203835.1_ASM20383v1_genomic.fna', format='fasta') if s.id == SC_ACCESSION
][0]

# write fasta file
SeqIO.write([sc_genome], os.path.join(temp, 'AL645882.2.fasta'), format='fasta')

1

In [4]:
%%bash
fasta-get-markov -m 1 $JUPYTER_TEMP/AL645882.2.fasta > $JUPYTER_TEMP/AL645882.2_m1.markov

cessed: 100.0%

### Prepare sequences
---
Extract sigE peak sequences

In [5]:
# load the SigE peaks
df_e = pd.read_excel('data/sige_macs_peaks_curated.xlsx', sheet_name='sigE_peaks')

In [6]:
df_e

,start,end,length,abs_summit,pileup,neglog10pvalue,fold_enrichment,neglog10qvalue,norm/EtOH,sequence_name
0,8662666,8663025,360,8662837,4159.26,399.36166,2.10648,397.12149,1,sigE_0
1,8483428,8484106,679,8483707,9107.42,2952.46606,4.61191,2948.32202,1,sigE_1
2,8038581,8038938,358,8038755,6019.20,1160.16821,3.04824,1157.51782,1,sigE_2
3,8005927,8006952,1026,8006657,9276.34,1427.76892,2.65750,1425.00598,1,sigE_3
4,7885747,7886423,677,7886038,8795.63,2617.73657,4.26193,2614.10645,1,sigE_4
...,...,...,...,...,...,...,...,...,...,...
188,6107752,6107920,169,6107856,7593.41,1068.68542,2.52894,1066.07446,1,sigE_188
189,5999380,5999532,153,5999460,6709.26,833.43805,2.37112,830.94513,x,sigE_189
190,4336288,4336472,185,4336360,7307.68,841.24341,2.28611,838.74652,1,sigE_190
191,1869170,1869322,153,1869226,9659.11,3326.14575,4.89125,3321.38403,1,sigE_191


In [7]:
features = pd.read_table(
    'data/GCA_000203835.1_ASM20383v1_genomic.gff', comment="#", names=['seqid', 'source', 'type', 'start', 'end', 'score', 'strand', 'phase', 'attributes']
)

features = features.loc[features.seqid == SC_ACCESSION]

def gff_attributes_mapper(gff_attr):
    d = dict()
    for attr in gff_attr.split(';'):
        k,v = attr.split('=')
        d[k] = v
    return d

features = pd.concat([features, pd.DataFrame(map(lambda x:x, features.attributes.apply(gff_attributes_mapper)))], axis=1)
features = features.drop(columns=['attributes'])

name_tag = 'gene'

f_fw = {k: v for k, v in features.loc[(features.strand == '+') & (~features[name_tag].isna()) & (features.type.isin({'CDS', 'gene'})), [name_tag, 'start']].values}
f_rc = {k: v for k, v in features.loc[(features.strand == '-') & (~features[name_tag].isna()) & (features.type.isin({'CDS', 'gene'})), [name_tag, 'end']].values}

# copy data set - retrieve data from source, set on new
df_sigE = df_e.copy()

### Run motif search
---

- using FIMO from MEME suite
- we use backgroud - first-order markov model from whole genome (AL645882.2)
- the individual hits will be processed


In [8]:
# write peak sequences

# use peak summit
summ_delta = 50
peak_seqs = []
for id, summ in df_sigE.loc[:, ['sequence_name', 'abs_summit']].values:
    s = summ-summ_delta
    e = summ+summ_delta
    peak_seqs.append(
        '>{}-{}-{}\n{}\n'.format(id, s, e, str(sc_genome.seq[s:e]))
    )
    
with open(os.path.join(temp, 'sigE_peaks_50.fasta'), 'w') as o:
    o.write('\n'.join(peak_seqs))
    
    
peak_seqs2 = []
for id, s, e in df_sigE.loc[:, ['sequence_name', 'start', 'end']].values:
    peak_seqs2.append(
        '>{}-{}-{}\n{}\n'.format(id, s, e-s+1, str(sc_genome.seq[s:e]))
   )
with open(os.path.join(temp, 'sigE_peaks_whole.fasta'), 'w') as o:
    o.write('\n'.join(peak_seqs2))
    
    
    
# use peak summit
summ_delta = 100
peak_seqs_100 = []
for id, summ in df_sigE.loc[:, ['sequence_name', 'abs_summit']].values:
    s = summ-summ_delta
    e = summ+summ_delta
    peak_seqs_100.append(
        '>{}-{}-{}\n{}\n'.format(id, s, e, str(sc_genome.seq[s:e]))
    )
with open(os.path.join(temp, 'sigE_peaks_100.fasta'), 'w') as o:
    o.write('\n'.join(peak_seqs_100))

### MEME
---

In [9]:
# MEME zoops with background, delta 50 nts

In [10]:
%%bash
meme $JUPYTER_TEMP/sigE_peaks_50.fasta -revcomp -bfile $JUPYTER_TEMP/AL645882.2_m1.markov -dna -mod zoops -minw 4 -maxw 7 -allw -oc $JUPYTER_TEMP/meme_zoops_test -nmotifs 10 -p 4

Writing results to output directory 'peak_summit_motif_discovery/meme_zoops_test'.
BACKGROUND: using background model of order 1
PRIMARY (classic): n 193 p0 193 p1 0 p2 0
SEQUENCE GROUP USAGE-- Starts/EM: p0; Trim: p0; pvalue: p0; nsites: p0,p1,p2
SEEDS: maxwords 19300 highwater mark: seq 193 pos 96
BALANCE: samples 193 chars 19300 nodes 4 chars/node 4825
Initializing the motif probability tables for 2 to 193 sites...
nsites = 192
Done initializing.
all widths from min to max

seqs=   193, min_w= 100, max_w=  100, total_size=    19300

motif=1
SEED DEPTHS: 2 4 8 16 32 64 128 193
SEED WIDTHS: 4 5 6 7
em: w=   7, psites=  32, iter=  40 
motif=2
SEED DEPTHS: 2 4 8 16 32 64 128 193
SEED WIDTHS: 4 5 6 7
em: w=   7, psites=  32, iter=  40 
motif=3
SEED DEPTHS: 2 4 8 16 32 64 128 193
SEED WIDTHS: 4 5 6 7
em: w=   7, psites=  32, iter=  40 
motif=4
SEED DEPTHS: 2 4 8 16 32 64 128 193
SEED WIDTHS: 4 5 6 7
em: w=   7, psites=  32, iter=  40 
motif=5
SEED DEPTHS: 2 4 8 16 32 64 128 193
SEED WIDTH

In [11]:
# MEME zoops, background, delta 100 nts

In [12]:
%%bash
meme $JUPYTER_TEMP/sigE_peaks_100.fasta -revcomp -bfile $JUPYTER_TEMP/AL645882.2_m1.markov -dna -mod zoops -minw 6 -maxw 10 -allw -oc $JUPYTER_TEMP/meme_zoops_test_100 -nmotifs 10 -p 4

Writing results to output directory 'peak_summit_motif_discovery/meme_zoops_test_100'.
BACKGROUND: using background model of order 1
PRIMARY (classic): n 193 p0 193 p1 0 p2 0
SEQUENCE GROUP USAGE-- Starts/EM: p0; Trim: p0; pvalue: p0; nsites: p0,p1,p2
SEEDS: maxwords 38600 highwater mark: seq 193 pos 194
BALANCE: samples 193 chars 38600 nodes 4 chars/node 9650
Initializing the motif probability tables for 2 to 193 sites...
nsites = 192
Done initializing.
all widths from min to max

seqs=   193, min_w= 200, max_w=  200, total_size=    38600

motif=1
SEED DEPTHS: 2 4 8 16 32 64 128 193
SEED WIDTHS: 6 7 8 9 10
em: w=  10, psites=  32, iter=  40 
motif=2
SEED DEPTHS: 2 4 8 16 32 64 128 193
SEED WIDTHS: 6 7 8 9 10
em: w=  10, psites=  32, iter=  40 
motif=3
SEED DEPTHS: 2 4 8 16 32 64 128 193
SEED WIDTHS: 6 7 8 9 10
em: w=  10, psites=  32, iter=  40 
motif=4
SEED DEPTHS: 2 4 8 16 32 64 128 193
SEED WIDTHS: 6 7 8 9 10
em: w=  10, psites=  32, iter=  40 
motif=5
SEED DEPTHS: 2 4 8 16 32 64 1

In [13]:
# MEME zoops, background, delta 100 nts, central distance as objective function

In [14]:
%%bash
meme $JUPYTER_TEMP/sigE_peaks_100.fasta -objfun cd -revcomp -bfile $JUPYTER_TEMP/AL645882.2_m1.markov -dna -mod zoops -minw 6 -maxw 10 -allw -oc $JUPYTER_TEMP/meme_zoops_test_100_cd -nmotifs 10 -p 4

Writing results to output directory 'peak_summit_motif_discovery/meme_zoops_test_100_cd'.
CD: cefrac 0.25 length 200 central region 50
BACKGROUND: using background model of order 1
PRIMARY (cd): n 193 p0 96 p1 48 p2 49
SEQUENCE GROUP USAGE-- Starts/EM: p0; Trim: p1; pvalue: p2; nsites: p0,p1,p2
SEEDS: maxwords 38600 highwater mark: seq 193 pos 194
BALANCE: samples 193 chars 38600 nodes 4 chars/node 9650
all widths from min to max

seqs=   193, min_w= 200, max_w=  200, total_size=    38600

motif=1
SEED DEPTHS: 2 4 8 16 32 64 96
SEED WIDTHS: 6 7 8 9 10
em: w=  10, psites=  32, iter=  30 
motif=2
SEED DEPTHS: 2 4 8 16 32 64 96
SEED WIDTHS: 6 7 8 9 10
em: w=  10, psites=  32, iter=  40 
motif=3
SEED DEPTHS: 2 4 8 16 32 64 96
SEED WIDTHS: 6 7 8 9 10
em: w=  10, psites=  32, iter=  40 
motif=4
SEED DEPTHS: 2 4 8 16 32 64 96
SEED WIDTHS: 6 7 8 9 10
em: w=  10, psites=  32, iter=  40 
motif=5
SEED DEPTHS: 2 4 8 16 32 64 96
SEED WIDTHS: 6 7 8 9 10
em: w=  10, psites=  32, iter=  40 
motif=6
SE

In [15]:
# MEME zoops, background, delta 50, motifsw 6-10

In [16]:
%%bash
meme $JUPYTER_TEMP/sigE_peaks_50.fasta -revcomp -bfile $JUPYTER_TEMP/AL645882.2_m1.markov -dna -mod zoops -minw 6 -maxw 10 -allw -oc $JUPYTER_TEMP/meme_zoops_test_6-10 -nmotifs 10 -p 4

Writing results to output directory 'peak_summit_motif_discovery/meme_zoops_test_6-10'.
BACKGROUND: using background model of order 1
PRIMARY (classic): n 193 p0 193 p1 0 p2 0
SEQUENCE GROUP USAGE-- Starts/EM: p0; Trim: p0; pvalue: p0; nsites: p0,p1,p2
SEEDS: maxwords 19300 highwater mark: seq 193 pos 94
BALANCE: samples 193 chars 19300 nodes 4 chars/node 4825
Initializing the motif probability tables for 2 to 193 sites...
nsites = 192
Done initializing.
all widths from min to max

seqs=   193, min_w= 100, max_w=  100, total_size=    19300

motif=1
SEED DEPTHS: 2 4 8 16 32 64 128 193
SEED WIDTHS: 6 7 8 9 10
em: w=  10, psites=  32, iter=  40 
motif=2
SEED DEPTHS: 2 4 8 16 32 64 128 193
SEED WIDTHS: 6 7 8 9 10
em: w=  10, psites=  32, iter=  40 
motif=3
SEED DEPTHS: 2 4 8 16 32 64 128 193
SEED WIDTHS: 6 7 8 9 10
em: w=  10, psites=  32, iter=  40 
motif=4
SEED DEPTHS: 2 4 8 16 32 64 128 193
SEED WIDTHS: 6 7 8 9 10
em: w=  10, psites=  32, iter=  40 
motif=5
SEED DEPTHS: 2 4 8 16 32 64 1

In [17]:
import meme_txt_parser
selected_motif = meme_txt_parser.read_file(os.path.join(temp, 'meme_zoops_test', 'meme.txt'))

# select motif MEME-1
selected_motif = selected_motif.loc[selected_motif.motif_id == "MEME-1", :]
seq_name = selected_motif.sequence_name.str.split('-')
selected_motif['sequence_name'] = [i[0] for i in seq_name]
selected_motif['sequence_offset'] = [int(i[1]) for i in seq_name]
selected_motif['stop'] = selected_motif.start + selected_motif.site.str.len()
selected_motif.drop(columns=['_site_left', '_site_right'], inplace=True)

In [18]:
selected_motif.head()

,motif_id,sequence_name,strand,start,P-value,site,sequence_offset,stop
0,MEME-1,sigE_181,+,16,0.000046,GAAGACG,2088569,23
1,MEME-1,sigE_171,+,53,0.000046,GAAGACG,3659392,60
2,MEME-1,sigE_168,+,81,0.000046,GAAGACG,4074167,88
3,MEME-1,sigE_156,-,43,0.000046,GAAGACG,6883700,50
4,MEME-1,sigE_154,+,51,0.000046,GAAGACG,7631695,58


In [19]:
# find direct downstream genes

features = pd.read_table(
    'data/GCA_000203835.1_ASM20383v1_genomic.gff', comment="#", names=['seqid', 'source', 'type', 'start', 'end', 'score', 'strand', 'phase', 'attributes']
)

features = features.loc[features.seqid == SC_ACCESSION]

def gff_attributes_mapper(gff_attr):
    d = dict()
    for attr in gff_attr.split(';'):
        k,v = attr.split('=')
        d[k] = v
    return d

features = pd.concat([features, pd.DataFrame(map(lambda x:x, features.attributes.apply(gff_attributes_mapper)))], axis=1)
features = features.drop(columns=['attributes'])

features_fw = features.loc[features.type.isin(['gene', 'CDS']) & (features.strand == '+'), :]
features_rc = features.loc[features.type.isin(['gene', 'CDS']) & (features.strand == '-'), :]


In [20]:
downstream_features = []
downstream_feature_distances = []
for r in selected_motif.itertuples():
    if r.strand == '+':
        feature_index = features_fw.start >= r.stop + r.sequence_offset
        if feature_index.any():
            downstream_feature = features_fw.loc[features_fw.loc[feature_index].start.idxmin()].to_dict()
            distance = downstream_feature['start'] - (r.stop + r.sequence_offset)
        else:
            downstream_feature = dict()
            distance = None
    elif r.strand == '-':
        feature_index = features_rc.end <= r.start + r.sequence_offset
        if feature_index.any():
            downstream_feature = features_rc.loc[features_rc.loc[feature_index].end.idxmax()].to_dict()
            distance = (r.start + r.sequence_offset) - downstream_feature['end']
        else:
            downstream_feature = dict()
            distance = None
    else:
        raise ValueError('Invalid strand')
    downstream_features.append(downstream_feature)
    downstream_feature_distances.append(distance)

downstream_features = pd.DataFrame(downstream_features)
downstream_features['distance'] = downstream_feature_distances
downstream_features.dropna(axis=1, how='all', inplace=True)

selected_motif_with_downstream = pd.concat([selected_motif, downstream_features.loc[:, ['gene', 'distance']]], axis=1)
selected_motif_with_downstream.rename(columns={'gene': 'downstream_gene', 'distance': 'distance_to_downstream'}, inplace=True)

In [21]:
selected_motif_with_downstream

,motif_id,sequence_name,strand,start,P-value,site,sequence_offset,stop,downstream_gene,distance_to_downstream
0,MEME-1,sigE_181,+,16,0.000046,GAAGACG,2088569,23,SCO1954,1617.0
1,MEME-1,sigE_171,+,53,0.000046,GAAGACG,3659392,60,SCO3310,2285.0
2,MEME-1,sigE_168,+,81,0.000046,GAAGACG,4074167,88,SCO3696,1683.0
3,MEME-1,sigE_156,-,43,0.000046,GAAGACG,6883700,50,SCO6255,5568.0
4,MEME-1,sigE_154,+,51,0.000046,GAAGACG,7631695,58,SCO6885,22393.0
...,...,...,...,...,...,...,...,...,...,...
82,MEME-1,sigE_116,-,42,0.000339,ATAGACG,3064610,49,SCO2806,88.0
83,MEME-1,sigE_178,-,53,0.000394,GTAGATG,2645931,60,SCO2458,563.0
84,MEME-1,sigE_158,+,49,0.000394,AAATAAG,6509941,56,SCO5943,28.0
85,MEME-1,sigE_187,-,36,0.000404,ATAGAAG,490356,43,SCO0468,42.0


In [22]:
with pd.ExcelWriter(os.path.join(temp, 'sigE_motif_discovery.xlsx'), engine='xlsxwriter') as writer:
    desc = [
        "Motif sites for MEME-1 motif discovered when searching for conserved 4-7nt mers in 100bp region of sigE peaks centered at abs_summit",
        "first order markov model of S.c. genome was used as background.",
        "Columns:",
        "  motif_id: meme assigned motif id",
        "  sequence_name: peak sequence id",
        "  strand: strand on which motif is found",
        "  start: motif site start position",
        "  P-value: motif site P-value (reported by meme)",
        "  site: discovered motif site sequence 5'->3'",
        "  sequence_offset: sequence start position in genome",
        "  stop: motif site stop position",
        "  downstream_gene: first gene downstream from motif stop",
        "  distance_to_downstream: distance from motif site to downstream gene [nt]"
    ]
    
    sheetname = 'sigE-motif'
    selected_motif_with_downstream.to_excel(writer, sheet_name=sheetname, startrow=len(desc) + 1, index=False)
    for i, row in enumerate(desc):
        writer.sheets[sheetname].write(i, 0, row)